In [1]:
import sys, os
sys.path.append("C:\\Program Files\\Lumerical\\v231\\api\\python\\")
import lumapi
# sys.path.append(os.path.dirname(__file__))

In [2]:
um = 1e-6
width = 0.5*um
height = 0.22*um
mode1 = lumapi.MODE()
rectangle = mode1.addrect(x = 0, y = 0.0, z = 0.0)
rectangle["x span"] = width
rectangle["y span"] = height
rectangle["z span"] = 10*um
rectangle['name'] = 'strip_waveguide'
rectangle['material'] = 'Si (Silicon) - Palik'

expansion_monitor = mode1.addfde()
expansion_monitor['x'] = 0
expansion_monitor['y'] = 0
expansion_monitor['z'] = 0
expansion_monitor['x span'] = 20*um
expansion_monitor['y span'] = 20*um
expansion_monitor['background material'] = 'SiO2 (Glass) - Palik'
expansion_monitor['start wavelength'] = 1.4e-6
expansion_monitor['start wavelength'] = 1.6e-6

mode1.switchtolayout()

# expansion_monitor.findmodes()
# expansion_monitor.run()
print(dir(expansion_monitor))
mode1.run()
print(dir(mode1))
# temp = mode1.selectmode(1)
# print(temp)

mode1.close()

['NA', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__self__', '__self_class__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__thisclass__', 'actual mesh cells x', 'actual mesh cells y', 'actual mesh cells z', 'add material gain to maximum tolerated gain', 'algorithm', 'allow symmetry on all boundaries', 'angle', 'automatically remove pml modes', 'background material', 'beam diameter', 'beam direction', 'beam radius', 'bend location', 'bend location x', 'bend location y', 'bend location z', 'bend orientation', 'bend radius', 'bent waveguide', 'calculate characteristic impedance', 'calculate group index', 'calculate impedance', 'center frequency', 'center the spectrum', 'center wavelength', 'center x', 'center y', 'center z', 'characteristic impedance', 'conformal